In [1]:
import snntorch as snn
import torch
from torchvision import datasets, transforms
from snntorch import utils

from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import matplotlib.pyplot as plt
from snntorch import spikegen
import numpy as np

dtype = torch.float
torch.set_default_dtype(dtype)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

c:\Users\dmartins\anaconda3\envs\mmSNN\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
audio_stim = np.load('auditory_stimuli.npz')
X_train_raw = audio_stim['X_train']
X_test_raw = audio_stim['X_test']
y_train = audio_stim['y_train']
y_test = audio_stim['y_test']

print(X_train_raw.shape, y_train.shape, X_test_raw.shape, y_test.shape)

(2100, 129, 81) (2100,) (900, 129, 81) (900,)


In [3]:
X_train = np.zeros([
    np.size(X_train_raw,0),
    np.size(X_train_raw,1) * np.size(X_train_raw,2)
])
X_test = np.zeros([
    np.size(X_test_raw,0),
    np.size(X_test_raw,1) * np.size(X_test_raw,2)
])

for i in range(np.size(X_train_raw,0)):
    X_train[i,:] = np.ravel(X_train_raw[i,:,:])

for i in range(np.size(X_test_raw,0)):
    X_test[i,:] = np.ravel(X_test_raw[i,:,:])

In [4]:
# Dataloader arguments
batch_size = 128*4

# Define a transform
transform = transforms.Compose([
    transforms.ToTensor()
])

training_dataset = TensorDataset(
    torch.from_numpy(X_train.astype(np.float32)),
    torch.from_numpy(y_train.astype(np.float32))
)
testing_dataset = TensorDataset(
    torch.from_numpy(X_test.astype(np.float32)),
    torch.from_numpy(y_test.astype(np.float32))
)

# Create DataLoaders
train_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(testing_dataset, batch_size=batch_size, shuffle=True, drop_last=True)


In [12]:
num_epochs = 30
loss_hist = []
test_loss_hist = []
counter = 0

# Define Network
class AudNet(nn.Module):
    def __init__(self,
        num_inputs=None, num_hidden=1000, num_last_hidden=20, num_output=10,
        beta=0.95, num_steps=81, num_freq=129):
        super().__init__()

        if num_inputs is None:
            num_inputs = num_freq

        self.num_steps = num_steps
        self.num_freq = num_freq

        # Initialize layers
        self.fc1 = nn.Linear(num_inputs, num_hidden)
        self.lif1 = snn.Leaky(beta=beta)
        self.fc2 = nn.Linear(num_hidden, num_hidden)
        self.lif2 = snn.Leaky(beta=beta)
        self.fc3 = nn.Linear(num_hidden, num_last_hidden)
        self.lif3 = snn.Leaky(beta=beta)
        self.fc4 = nn.Linear(num_last_hidden, num_output)
        self.lif4 = snn.Leaky(beta=beta)

    def forward(self, x):

        x.to(torch.float32)

        # Initialize hidden states at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        mem3 = self.lif3.init_leaky()
        mem4 = self.lif3.init_leaky()

        output_spike_record = []
        output_memV_record = []

        for step in range(self.num_steps):

            x_ = x.reshape(-1, self.num_freq, self.num_steps)[:,:,step]

            cur1 = self.fc1(x_)
            spk1, mem1 = self.lif1(cur1, mem1)

            cur2 = self.fc2(spk1)
            spk2, mem2 = self.lif2(cur2, mem2)

            cur3 = self.fc3(spk2)
            spk3, mem3 = self.lif3(cur3, mem3)

            cur4 = self.fc4(spk3)
            spk4, mem4 = self.lif4(cur4, mem4)

            output_spike_record.append(spk4)
            output_memV_record.append(mem4)

        self.mem1.append(torch.stack(mem1))
        self.mem2.append(torch.stack(mem2))
        self.mem3.append(torch.stack(mem3))

        return  torch.stack(output_spike_record, dim=0), torch.stack(output_memV_record, dim=0)

    def fwd_frozen(self, x):

        mem1 = self.mem1
        mem2 = self.mem2
        mem3 = self.mem3

        last_hidden_spike_record = []
        last_hidden_output_memV_record = []

        for step in range(self.num_steps):

            x_ = x.reshape(self.num_freq, self.num_steps)[:,step]

            cur1 = self.fc1(x_)
            spk1, mem1 = self.lif1(cur1, mem1)

            cur2 = self.fc2(spk1)
            spk2, mem2 = self.lif2(cur2, mem2)

            cur3 = self.fc3(spk2)
            spk3, mem3 = self.lif3(cur3, mem3)

            last_hidden_spike_record.append(spk3)
            last_hidden_output_memV_record.append(mem3)

        return torch.stack(last_hidden_spike_record, dim=0), torch.stack(last_hidden_output_memV_record, dim=0)


In [13]:
# input_sz = np.size(X_train, 1)
# Load the network onto CUDA if available
net = AudNet().to(device)

# pass data into the network, sum the spikes over time
# and compare the neuron with the highest number of spikes
# with the target

def print_batch_accuracy(data, targets, train=False):
    output, _ = net(data.view(batch_size, -1))
    _, idx = output.max(1)
    acc = np.mean((targets == idx).detach().cpu().numpy())

    # if train:
    #     print(f"Train set accuracy for a single minibatch: {acc*100:.2f}%")
    # else:
    print(f"Test set accuracy for a single minibatch: {acc*100:.2f}%")

def train_printer():
    print(f"Epoch {epoch}, Iteration {iter_counter}, Train loss = {loss_hist[counter]:.2f} Test loss = {test_loss_hist[counter]:.2f} \n")
    print_batch_accuracy(data, targets)
    print('\n')

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=5e-4, betas=(0.9, 0.999))

# Outer training loop
for epoch in range(num_epochs):
    iter_counter = 0
    train_batch = iter(train_loader)

    # Minibatch training loop
    for data, targets in train_batch:
        data = data.to(device)
        targets = targets.to(device)

        # forward pass
        net.train()
        spk_rec, mem_rec = net(data.view(batch_size, -1))

        # initialize the loss & sum over time
        # loss_val = torch.zeros((1), dtype=dtype, device=device)
        loss_val = loss(mem_rec, targets.type(torch.long))

        # Gradient calculation + weight update
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()

        # Store loss history for future plotting
        loss_hist.append(loss_val.item())

        # Test set
        with torch.no_grad():
            net.eval()
            test_data, test_targets = next(iter(test_loader))
            test_data = test_data.to(device)
            test_targets = test_targets.to(device)

            # Test set forward pass
            test_spk, test_mem = net(test_data.view(batch_size, -1))

            # Test set loss
            # test_loss = torch.zeros((1), dtype=dtype, device=device)
            # for step in range(num_steps):
            test_loss = loss(test_mem, test_targets.type(torch.long))
            test_loss_hist.append(test_loss.item())

            # Print train/test loss/accuracy
            if counter % 50 == 0:
                train_printer()
            counter += 1
            iter_counter +=1

In [8]:
data.view(batch_size, -1).shape

torch.Size([512, 10449])

In [15]:
data.view(batch_size, -1).shape

torch.Size([512, 10449])

In [14]:
129*81

10449

In [ ]:
output, _ = net(data.view(batch_size, -1))
output.shape

In [ ]:
_, idx = output.max(1)
# acc = np.mean((targets == idx).detach().cpu().numpy())

In [ ]:
test_mem.shape, test_targets.type(torch.long).shape

In [ ]:
X_train.shape

In [ ]:
from scipy.signal import spectrogram

In [ ]:
_, _, s = spectrogram(X_train[2,:], fs=8000)

In [ ]:
X_train[0,:].shape

In [ ]:
plt.plot(X_train[2,:])

In [ ]:
s.shape

In [ ]:
plt.imshow(s)
plt.xlim([0,20])
plt.ylim([0,20])

In [ ]:
plt.plot(X_train[1,:], 'k')
plt.xlim([0,2400])
plt.xlabel('samples')

In [ ]:
print(data.shape, targets.shape)

In [ ]:
mem_rec.shape

In [ ]:
test_mem.shape

In [ ]:
test_targets.shape

In [ ]:
spk_rec, mem_rec = net(data.view(batch_size, -1))
spk_rec.shape, mem_rec.shape

In [ ]:
for param in net.parameters():
    print(param.dtype)